In [ ]:
import os
import re
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

# Load environment variables
load_dotenv()


# Initialize Watsonx LLM
def watsonx_llm(messages):
    model_id = "meta-llama/llama-3-1-8b-instruct"
    api_key = os.getenv("WX_KEY")
    url = os.getenv("WX_URL")
    project_id = os.getenv("WX_PID")

    if not all([api_key, project_id, model_id]):
        raise Exception(
            "Missing required environment variables: WX_KEY, WX_URL, or WX_PID."
        )

    params = {
        GenParams.DECODING_METHOD: "sample",
        GenParams.MAX_NEW_TOKENS: 256,
        GenParams.TEMPERATURE: 0.0,
        GenParams.STOP_SEQUENCES: ["<|eot_id|>", "<|eom_id|>"],
    }

    credentials = Credentials(api_key=api_key, url=url)
    model_inference = ModelInference(
        model_id=model_id, params=params, credentials=credentials, project_id=project_id
    )
    response = model_inference.chat(messages=messages)
    return response["choices"][0]["message"]["content"]


# Preprocess Documents for RAG
loader = DirectoryLoader(
    "/Users/charan/VSCode/EMEA/Comarch_Telco_OSS_Assistant/data",
    glob="**/*.pdf",
    loader_cls=PyMuPDFLoader,
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=50
)
split_docs = text_splitter.split_documents(docs)

# Embedding and Vector Store
embeddings = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url=os.getenv("WX_URL"),
    apikey=os.getenv("WX_KEY"),
    project_id=os.getenv("WX_PID"),
)
vectorstore = Chroma.from_documents(
    split_docs, collection_name="agentic-rag-chroma", embedding=embeddings
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})


# Define Retrieval Tool
def get_context(question):
    results = retriever.invoke(question)
    return "\n".join([doc.page_content for doc in results])


# Define available actions
known_actions = {
    "get_context": get_context,
}


# Define ReAct Agent
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        return watsonx_llm(self.messages)


# Initialize the prompt
prompt = """
You operate as a retrieval-augmented assistant with tools to fetch information from documents.
You follow this loop:

1. Thought: Describe your reasoning process.
2. Action: Run a tool to get additional context or answer the question.
3. Observation: Report the results of the action.
4. Answer: Use the retrieved information and your reasoning to answer.

Available actions:

get_context:
e.g., get_context: What is the definition of Generic Network Slice Template?
Fetches context about the given question from stored documents.
""".strip()

# Define the query function with ReAct loop
action_re = re.compile(r"^Action: (\w+): (.*)$")


def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f" -- running {action} {action_input}")
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result


# Example Usage
question = "What is Generic Network Slice Template as defined by GSMA?"
query(question)

In [ ]:
# Example Usage
question = "What is Generic Network Slice Template as defined by GSMA?"
query(question)